<a href="https://colab.research.google.com/github/adrian-alejandro/BDMA/blob/main/proyecto/visualization/Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project proposal
## Master Big Data Management, Technologies and Analytics - Visualization | 2023 - 2024
### Integrantes: Genís Barberà i Gutiérrez - Adrián Zelaya - Leonardo Cortés

## Fuentes de datos:
Para este trabajo hemos hecho uso de los siguientes datasets.
### Parliament Representation:
*   [Seats held by women in national parliaments and governments](https://ec.europa.eu/eurostat/databrowser/view/sdg_05_50/default/table?lang=en)

### Violence:
*   [Physical and sexual violence to women by age group (2012 data)](https://ec.europa.eu/eurostat/databrowser/view/sdg_05_10/default/table?lang=en)

*   [Women who have experienced violence by any perpetrator, by age group (2021 data)](https://ec.europa.eu/eurostat/databrowser/view/gbv_any_age/default/table?lang=en)

### HDI:
* [Human Development Index- ourworldindata](https://ourworldindata.org/grapher/human-development-index?tab=table#reuse-this-work)


### GDP:
* [Real GDP Per Capita](https://ec.europa.eu/eurostat/databrowser/view/sdg_08_10/default/table)

## Configuración de entorno

Nos aseguramos de tener la última versión disponible de Altair, dado que sino hay algunas funcionalidades que fallan.

In [ ]:
!pip install -U "altair[all]" vega_datasets

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import altair as alt
from os import path
from vega_datasets import data
from altair import datum

## Ingestión y procesamiento de los datos

In [ ]:
# DATASETS PROCESADOS
DATASETS = ['Parliament_processed.csv', 'GDP_processed.csv', 'HDI_processed.csv', 'Violence_processed.csv']

# Los siguientes archivos de Excel serán necesarios para la ingesta y procesamiento de los datos,
# no es necesario que estén disponible en el entorno de Colab, dado que en caso de falla, toma los datos de un repositorio de GitHub
PARLIAMENT_DATA = 'sdg_05_50_spreadsheet.xlsx'
GDP_DATA = 'GDP_sdg_08_10_spreadsheet.xlsx'
HDI_DATA = 'human-development-index.csv'
VIOLENCE_DATA_2021 = 'gbv_any_age_page_spreadsheet.xlsx'
VIOLENCE_DATA_2012 = 'genvio_phy_oth_sur__prev_phys_sex__ipv_grand.xlsx'
VIOLENCE_2012_FILES = ['genvio_phy_oth_sur__prev_phys_sex__ipv_grand_Total.xlsx',
                       'genvio_phy_oth_sur__prev_phys_sex__ipv_grand_18-29.xlsx',
                       'genvio_phy_oth_sur__prev_phys_sex__ipv_grand_30-39.xlsx',
                       'genvio_phy_oth_sur__prev_phys_sex__ipv_grand_40-49.xlsx',
                       'genvio_phy_oth_sur__prev_phys_sex__ipv_grand_50-59.xlsx',
                       'genvio_phy_oth_sur__prev_phys_sex__ipv_grand_60+.xlsx']

# Definición de los países de la UE, que nos será útil para filtrar posteriormente
EU_COUNTRIES = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',
                'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
                'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
                'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
                'Slovenia', 'Spain', 'Sweden']
# Para descarga de datos desde GitHub (uso opcional)
URL_GITHUB = 'https://github.com/adrian-alejandro/BDMA/raw/main/proyecto/visualization/data'

### Funciones
Definimos las siguientes funciones para la carga y procesamiento de los datos. Las descripciones específicas del procesamiento de cada dataset se encuentran entre las líneas de código.

In [ ]:
def load_data(filename, header_row = None, url_path=URL_GITHUB, sheet_name='Sheet 1'):
  """Función que carga los datos desde Colab o GitHub, y devuelve un dataframe.
  Se asume que la data se ha subido a Colab, o en su defecto a un repositorio de GitHub si no están disponibles en Colab.
  """
  if path.exists(filename):
    file_path = filename
  else:
    # Si el fichero no se encuentra disponible en Colab, lo cargamos desde un repositorio en GitHub
    file_path = path.join(url_path, filename)

  try:
    # Cargar los datos desde el archivo Excel, con encabezados en la fila correcta
    if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
      return pd.read_excel(file_path,
                         sheet_name=sheet_name,
                         header=header_row)
    elif file_path.endswith('.csv'):
      # cargar los datos desde el archivo csv
      return pd.read_csv(file_path, header=header_row) if header_row is not None else pd.read_csv(file_path)
  except Exception as e:
    print("Error en la carga del fichero, asegurarse de que el archivo Excel esté disponible en el entorno de Colab", e)
    return None

def process_hdi(dataset):
  """Función que carga los datos de HDI, renombra algunas columnas y filtra por año para que el dataset pueda ser usado con los otros
  """
  df = load_data(dataset)
  df = df.rename(columns={'Entity':'Region', 'Year':'Año', 'Human Development Index':'HDI'})
  df_filtro_regiones = df[df['Region'].isin(EU_COUNTRIES)]
  df_final = df_filtro_regiones[df_filtro_regiones['Año']>2002]
  return df_final


def load_and_preprocess_data(filename, header_row, year_to_numeric=False,
                             melt_column=None):
  """Función que carga los datos y realiza el procesamiento de base de los datasets.
  Se asume que la data se ha subido a Colab, o en su defecto a un repositorio de GitHub si no están disponibles en Colab.
  """
  # Cargamos la data
  df = load_data(filename, header_row)

  # Renombrar la primera columna para usarla como identificador de la región
  df.rename(columns={df.columns[0]: 'Region'}, inplace=True)

  # Descartamos las columnas que están de más
  df = df.drop(columns=[x for x in df.columns if ':' in x])

  # Descartamos las filas con nulos (antes hemos verificado que no haya nulos en filas con data)
  df = df[~(df.isnull().sum(axis = 1) > 0)].reset_index(drop=True)

  # Comprimimos/fundimos todas las columnas (salvo la primera) a una sola cuyo target column name es 'melt_column'
  if melt_column:
    # Seleccionar solo las columnas de años que contienen datos, excluyendo las marcas 'd'
    year_columns = df.columns[1:].tolist()

    # Transformar los datos para análisis
    value_name = 'GDP' if 'GDP' in filename else 'Porcentaje'
    df = df.melt(id_vars=['Region'], value_vars=year_columns, var_name=melt_column, value_name=value_name)

    if year_to_numeric:
      # Convertir la columna 'Año' a numérico
      df[melt_column] = pd.to_numeric(df[melt_column], errors='coerce')

  # Convertir la columna 'Porcentaje' a numérico, manejando los valores no disponibles ':'
  df[value_name] = pd.to_numeric(df[value_name].replace(':', None), errors='coerce')

  # Agregamos la columna 'Grupo', que distingue si es EU o Europa (no-EU)
  df = filter_eu_countries(df, 'Region')

  return df


def load_and_process_parliament_data(filename, header_row=9, year_to_numeric=True,
                                     melt_column='Año'):
  """Función que realiza las transformaciones específicas del dataset de Women Representation in EU Parliaments
  """
  # Cargamos los datos
  df = load_and_preprocess_data(filename, header_row=header_row,
                                year_to_numeric=year_to_numeric, melt_column=melt_column)
  # Realizamos una copia del dataset y calculamos gap
  df_m = df.copy()
  df_m['Gap'] = 100 - 2*df_m['Porcentaje']
  # Realizamos copia para asignar al género hombre
  df_h = df_m.copy()
  # Agregamos la columna Género, así podemos armar un Ranged Dot Plot
  df_m['Género'] = 'Mujer'
  df_h['Género'] = 'Hombre'
  # Establecemos el porcentaje correspondiente a los hombres
  df_h['Porcentaje'] = 100 - df_m['Porcentaje']
  # Devolvemos ambos subsets concatenados
  return pd.concat([df_m, df_h], axis=0).reset_index(drop=True)

def load_and_process_GDP_data(filename, header_row=8, year_to_numeric=True,
                                     melt_column='Año'):
  """Función que realiza las transformaciones específicas del dataset de GDP
  """
  # Cargamos los datos
  df = load_and_preprocess_data(filename, header_row=header_row,
                                year_to_numeric=year_to_numeric, melt_column=melt_column)

  return df.reset_index(drop=True)

def load_and_process_violence_data_2012(files=None, filename=None, output_file=None):
  """Función que toma archivos de subsets de data y los concatena en uno.
  Alternativamente, lee el archivo ya procesado y devuelve un dataframe
  """
  if files:
    dataframes = []

    for file in files:
      # Obtenemos los valores de rango etario en base al nombre del fichero
      rango = file.split('_')[-1].split('.')[0]

      # Leemos los ficheros (de a uno)
      aux_df = load_data(file, header_row=19, sheet_name='Data',
                          url_path=path.join(URL_GITHUB, 'violence-2012'))

      # Cada columna se corresponde con el valor de % de violencia (física y/o sexual)
      # según dos niveles de segmentación:
      # 1) tipo de período de prevalencia (último año o toda la vida),
      # 2) tipo de perpetrador de la violencia
      # Limpiamos nombres de columnas, para luego obtener el tipo de período
      columns = [x.split('.')[0] for x in aux_df.columns.values]

      aux_dfs = []
      period_col_name = columns[0]

      # La primera fila corresponde al nombre de las columnas correspondientes
      # al nivel del segmento de tipo de perpetradores
      # De la primera celda obtenemos los nombres de las columnas que usaremos
      region_col_name, perpetrator_col_name = aux_df.iloc[0, 0].split('\\')

      # Luego, iteramos por cada columna para fundir los datos en menos columnas,
      # así se puede trabajar mejor y es consistente con el dataset de 2021
      for i, period in enumerate(columns):
        if i > 0:
          _aux_df = aux_df.iloc[1:, [0]].copy()
          # Renombramos la primera columna
          _aux_df.columns = [region_col_name.split(' ')[-1].title()]
          # Agregamos las columnas que necesitamos
          _aux_df[period_col_name] = columns[i]
          _aux_df[perpetrator_col_name] = aux_df.iloc[0, i]
          _aux_df['Porcentaje'] = aux_df.iloc[1:, i]

          aux_dfs.append(_aux_df)

      # Concatenamos los datos
      age_range_df = pd.concat(aux_dfs, axis=0).reset_index(drop=True)
      # agregamos el rango etario correspondiente al fichero procesado
      age_range_df['Edad'] = rango
      dataframes.append(age_range_df)

    # Concatenamos una vez más, para tener todos los rangos etarios en el mismo lugar
    df_violence = pd.concat(dataframes)

    # Agregamos el año
    df_violence['Año'] = 2012

    # Agregamos columnas Grupo y Entidad
    df_violence = filter_eu_countries(df_violence, 'Region')

    if output_file:
      # Exportamos a un archivo
      df_violence.to_excel(output_file, index=False)

  else:
    df_violence = pd.read_excel(filename)

  return df_violence


def load_and_process_violence_data(filename_2021, filename_2012=None, files_2012=None):
  """Función que realiza las transformaciones específicas del dataset de Women who have experienced violence by any perpetrator, by age group
  """
  # Cargamos los datos del 2021
  df_2021 = load_and_preprocess_data(filename_2021, header_row=8, melt_column='Edad')
  # Agregamos el año
  df_2021['Año'] = 2021
  df_2021['Year'] = 2021
  # Modificamos el formato de los valores del campo edad para que sea consistente con el 2012 dataset
  df_2021['Edad'] = df_2021['Edad'].str.replace(' to ', '-')\
                                   .str.replace(r'[a-zA-Z ]', '', regex=True)
  # Renombramos el rango 18-74 como 'Total', también para que sea consistente con el 2012 dataset
  df_2021['Edad'] = df_2021['Edad'].apply(lambda x: 'Total' if x == '18-74' else x, 1)

  # Cargamos los datos del 2012
  df_2012 = load_and_process_violence_data_2012(filename=filename_2012,
                                                files=files_2012,
                                                output_file=filename_2012)
  df_2012['Year'] = 2012
  # Filtramos los datos del 2012 para que coincidan con el 2021,
  # i.e. 1) Type of perpetrator: Any, Period covered: Last 12 months
  filter = (df_2012['Type of perpetrator'].str.contains('Any')) & \
          (df_2012['Period covered'].str.contains('12'))
  # Aplicamos filtro y seleccionamos solo las columnas en común entre ambos datasets
  df_2012 = df_2012[filter].loc[:, df_2021.columns.tolist()]

  # Concatenamos los datasets
  df = pd.concat([df_2012, df_2021], axis=0).reset_index(drop=True)

  # Duplicamos la columna "Porcentaje" para ordernar los datos en las visualizaciones
  df['PorcentajeSort'] = df['Porcentaje']

  # Realizamos una copia del dataset
  df_v = df.copy()
  # Realizamos copia para etiquetar los casos de no-violencia
  df_nv = df.copy()
  # Agregamos la columna Violencia, así nos facilita el Normalized Stacked Plot
  df_v['Violencia'] = 'Sí'
  df_nv['Violencia'] = 'No'
  # Establecemos el porcentaje correspondiente a los casos de no-violencia
  df_nv['Porcentaje'] = 100 - df_nv['Porcentaje']

  # Devolvemos ambos subsets concatenados
  return pd.concat([df_v, df_nv], axis=0).reset_index(drop=True)


def filter_eu_countries(dataframe, subject_column, eu_countries=EU_COUNTRIES):
  """Función que agrega dos columnas al dataframe:
   1) clasificación como EU / non-EU
   2) nombre abreviado de la entidad/país
  """
  df = dataframe.copy()
  # Agregamos la columna 'Grupo', que distingue si es EU o Europa (no-EU)
  df['Grupo'] = df.apply(lambda x: 'EU' if (x[subject_column] in eu_countries) or ('European Union' in x[subject_column])
                                        else 'Europa (no-EU)', 1)
  df['Entidad'] = df.apply(lambda x: "European Union" if 'European Union' in x[subject_column] else x[subject_column], 1)

  return df


# Definimos la siguiente función lamdba para facilitar el filtrado de los datos,
# donde seleccionamos los valores correspondient'es a EU + países individuales de la EU.
# Dado que hay dos series de datos para la EU: 28 países (hasta 2020) y 27 países (todo el rango), optamos por utilizar EU 27
filter_eu = lambda df: (df['Grupo'] == 'EU') & ~(df['Region'].str.contains('28'))

### Cargamos y procesamos los datos

In [ ]:
# Flag para determinar si procesamos los datos o si tomamos la data procesada
process_data = False
if process_data:
  df_parliament = load_and_process_parliament_data(PARLIAMENT_DATA)
  df_gdp = load_and_process_GDP_data(GDP_DATA)
  df_hdi = process_hdi(HDI_DATA)
  df_violence = load_and_process_violence_data(VIOLENCE_DATA_2021,
                                              filename_2012=VIOLENCE_DATA_2012,
                                              files_2012=VIOLENCE_2012_FILES)
else:
  df_parliament = pd.read_csv(DATASETS[0])
  df_gdp = pd.read_csv(DATASETS[1])
  df_hdi = pd.read_csv(DATASETS[2])
  df_violence = pd.read_csv(DATASETS[3])

## Visualizaciones

In [ ]:
# Establecemos algunas variables de dimensión que nos serán útiles
LEFT_CHART_WIDTH = 600
RIGHT_CHART_WIDTH = 800
CHART_HEIGHT = 600
LABEL_FONT_SIZE = 14

# Condiciones de formateo que nos ayudará a tener un código más legible
formato = lambda cond, true, false: (cond, alt.value(true), alt.value(false))

### Representación de mujeres en Parlamentos de la UE

Para responder las preguntas armamos dos charts concatenados horizontalmente:
* El primero es un *Ranged Dot Plot*, que ayuda a visualizar el gap por año. Para lograr una transición más ágil entre año a año, agregamos un slider que permite seleccionar el año. Este permite responder las preguntas: **"Exists a gender gap in EU Parliaments?"** y **"How it exists, how big is the difference?"**
* El segundo es un *Line Plot* complementario, el cual brinda toda la evolución del % de representación para el período disponible de datos. Este permite responder la pregunta: **"Has the difference changed along the last 24 years?"**

Primero seleccionamos el subset de datos que vamos a incluir en las visualizaciones, luego procedemos a definir el rango temporal en base a los datos.




In [ ]:
# Seleccionamos los valores correspondientes a EU, países individuales de la EU
# Dado que hay dos series de datos para la EU: 28 países (hasta 2020) y 27 países (todo el rango), optamos por utilizar EU 27
source = df_parliament[filter_eu(df_parliament)]

# Valores que nos serań útiles
min_yr = source['Año'].min()
max_yr = source['Año'].max()

#### **Ranged Dot Plot:** *Brecha de género en la representación de mujeres en parlamentos de la UE*

En este gráfico quisimos mostrar en simultáneo el porcentaje de representación de mujeres/hombres en parlamentos europeos y la diferencia (gap) entre ambos, para un dado año. Para ello utilizamos un ranged dot plot.

Para facilitar la interacción del usuario con el gráfico agregamos las siguientes funcionalidades:
- Barra deslizadora de años (slider), que permite seleccionar dinámicamente el año de consulta
- Resaltador de selección de país, que permite resaltar y agrandar el gap que se encuentra inmediatamente debajo del cursor del mouse.

Para visualizar mejor la tendencia, hemos ordenado los puntos de acuerdo al valor de gap entre % de representación entre hombres y mujeres.

Utilizamos colores neutrales, basándonos en paletas mencionadas en el siguiente [artículo](https://blog.datawrapper.de/gendercolor/).

Para ayudar a la interpretación se han añadido 'tooltips' con la información correspondiente a cada punto, como ser País, % representación, gap, año.

In [ ]:
#### RANGED DOT PLOT ####

# Definimos los selectores de nuestro gráfico
# Armamos un slider que nos permita seleccionar dinámicamente el año
slider = alt.binding_range(min=min_yr, max=max_yr, step=1,
                           name="Seleccione el año de visualización para el ranged dot plot: ")

# Objeto de selección de un punto en base al año
select_year = alt.selection_point(name='Año', fields=['Año'],
                                   bind=slider, value=max_yr)

# Objeto de selección de punto en base al país seleccionado en la leyenda
select_country = alt.selection_point(fields=['Entidad'], bind='legend')

# Efecto sobre la visualización para resaltar el punto de interés
highlight = alt.selection_point(
    on="mouseover", fields=["Entidad"], nearest=True, empty='none'
)

# Armamos la base de nuestro chart
chart = alt.Chart(
    source,
    title=alt.Title(
       "Proporción de representantes en Parlamentos de la UE por género y país",
       subtitle="Ordenados ascendentemente por diferencia de representación (brecha)",
       # Agrandamos el fontsize para que sea más legible
       fontSize=LABEL_FONT_SIZE + 2
    )
).transform_filter(
    filter={'field': 'Género',
            "oneOf": ['Mujer', 'Hombre']}
).add_params(
    select_year,
    select_country
).transform_filter(
    select_year
).transform_filter(
    select_country
).transform_filter(
    'isValid(datum.Porcentaje)'
).transform_calculate(
    Gap_text='datum.Entidad + \' Gap: \' + format(datum.Gap, ".2f") + "%"'
)

# Definimos las líneas que representan el gap entre los valores de mujeres y varones
gap = chart.mark_line().encode(
    x=alt.X('Porcentaje:Q',
            scale=alt.Scale(domain=[0, 100]), ),
    y=alt.Y('Entidad:N', title="",
            # Agrandamos el fontsize para que sea más legible
            axis=alt.Axis(labelFontSize=LABEL_FONT_SIZE, titleFontSize=LABEL_FONT_SIZE + 2),
            sort=alt.EncodingSortField(field='Gap', order='ascending')
            ),
    color=alt.value('#5E666E'),
    detail='Region:N',
    strokeWidth=alt.condition(highlight, alt.value(10), alt.value(5)),
    strokeOpacity=alt.condition(highlight, alt.value(1), alt.value(0.5))
)

# Definimos los puntos de los porcentajes de representación por género
points = chart.mark_point(
    filled=True
).encode(
    x=alt.X('Porcentaje:Q', title="Proporción de representantes por género (%)",
            # Fijamos la escala para que se mantenga constante independiente de los valores anuales
            scale=alt.Scale(domain=[0, 100]),
            # Agrandamos el fontsize para que sea más legible
            axis=alt.Axis(labelFontSize=LABEL_FONT_SIZE, titleFontSize=LABEL_FONT_SIZE + 2),
            ),
    y=alt.Y('Entidad:N', title="",
            # Ordenamos el eje Y de acuerdo al gap de representantes por sexo
            sort=alt.EncodingSortField(field='Gap', order='ascending'),
            ),
    # Determinamos los colores en base al género y ubicamos la legenda debajo del título para mayor visibilidad y aprovechamiento del espacio
    color=alt.Color('Género:O',
                    # Ubicamos la leyenda DENTRO del gráfico para un uso más eficiente del espacio,
                    # sabiendo que en ningún momento se solapará con los datos
                    legend=alt.Legend(
                        fillColor='#EEEEEE', orient='top-right',
                        padding=10, cornerRadius=10, labelFontSize=14, titleFontSize=14
                        )
                    ).scale(
                        domain=['Mujer', 'Hombre'],
                        # Utilizamos un color gender-neutral
                        range=['#0D6D64', '#F4BA3B']
                        ),
    # Formateamos el tamaño y opacidad de los markers dependiendo de si asomamos el mouse/pointer o si seleccionamos un país
    size=alt.condition(highlight, alt.value(500), alt.value(100)),
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.75)),
    # Incluimos un tooltip con la información de cada punto
    tooltip=['Género:N', 'Entidad:N', 'Porcentaje:Q', 'Gap:Q', 'Año:N']
).properties(
    width=LEFT_CHART_WIDTH,
    height=CHART_HEIGHT
).add_params(
    highlight
)

text = chart.mark_text(align='left', dx=10, dy=0, fontSize=15).encode(
    text=alt.condition(highlight, 'Gap_text:N', alt.value('')),
    x=alt.value(10),
    y=alt.value(10)
).transform_filter(
    highlight
)

ranged_dot_plot = alt.layer(gap, points, text).resolve_scale(
    color='independent',
    size='independent',
    opacity='independent'
)

ranged_dot_plot


/usr/local/lib/python3.10/dist-packages/altair/vegalite/v5/api.py:398: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(


alt.LayerChart(...)

#### **Line Plot:** *Evolución temporal de representación de mujeres en palamentos de la UE*

En este gráfico buscamos mostrar la evolución temporal de la representación de asientos de mujeres en parlamentos, como una medida equivalente del gap ya que están directamente correlacionadas (cuanto más cerca del 50%, menor gap).

En este caso optamos por mantener un único color para representar dicha evolución: el mismo asignado al sexo Mujer en el ranged dot plot, independientemente del país seleccionado. Esto es para evitar confusiones en el usuario, así no tiene que dedicar mucho tiempo en pensar a qué corresponde el color. Los países que se encuentran fuera de selección se muestran con una transparencia así no distraen.

Las selecciones de este gráfico están coordinadas con el gráfico anterior con el fin de mejorar la experiencia de usuario.

In [ ]:
#### LINE PLOT ####

# Definimos el color que utilizaremos para el line plot. Dado que solamente
# presentaremos información correspondiente a mujeres, forzamos un único color
# (consistente con el gráfico anterior)
color = alt.Color('Entidad:N', legend=alt.Legend(
                                title="Seleccione país para filtrar: ",
                                symbolType="square", symbolOpacity=0, symbolSize=0,
                                titlePadding=10, labelOffset=0, titleFontSize=10)
).scale(
    # Aquí definimos una paleta consistente en un único color, el asignado al género Mujer
    range=['#0D6D64']
)

# Definimos un selector de intervalo (brush) para resaltar un área de interés
brush = alt.selection_interval(encodings=['x'])

# Definimos las líneas que representan % mujeres de un país seleccionado over time
# Dentro de la definición del gráfico de líneas (line_plot)

lines = alt.Chart(source).mark_line(
    point=alt.OverlayMarkDef(filled=False, fill="white")
).encode(
    x=alt.X('Año:N',
            axis=alt.Axis(# Rotamos labels y agrandamos el fontsize para que sea más legible
                labelAngle=-40,
                labelFontSize=LABEL_FONT_SIZE,
                titleFontSize=LABEL_FONT_SIZE + 2)
    ),
    y=alt.Y('Porcentaje:Q',
            # Agrandamos el fontsize para que sea más legible
            axis=alt.Axis(labelFontSize=LABEL_FONT_SIZE, titleFontSize=LABEL_FONT_SIZE + 2)
    ),
    color=color,
    # Condicionamos el grosor y opacidad de las líneas según estén seleccionadas/hovered o no
    strokeWidth=alt.condition(*formato(highlight, 3, 1)),
    strokeOpacity=alt.condition(*formato(highlight, 1, 0.5))
).add_params(
    brush,
    select_country  # Añadir selección interactiva aquí
).transform_filter(
    brush
).transform_filter(
    alt.FieldEqualPredicate(field='Género', equal='Mujer')
).transform_filter(
    select_country  # Aplicar filtro de selección aquí
).properties(
    width=RIGHT_CHART_WIDTH,
    height=CHART_HEIGHT,
    title=alt.Title(f'Evolución porcentaje de mujeres en Parlamentos de la UE ({min_yr}-{max_yr})', fontSize=LABEL_FONT_SIZE+2)
)

# Armamos legends & entity_name para agregar leyendas al final de cada línea,
# así es más fácil identificarlas
legends = lines.mark_circle().encode(
    alt.X("last_year['Año']:N"),
    alt.Y("last_year['Porcentaje']:Q")
)

entity_name = legends.mark_text(align="left", dx=10, size=14).encode(text="Entidad")

# Situamos los plots en la misma capa y unificamos nombres de ejes
line_plot = (lines + entity_name).encode(
    x=alt.X().title("Año"),
    y=alt.Y().title("Proporción de representantes mujeres (%)")
)

#### Concatenación de gráficos

In [ ]:
#### CONCATENACIÓN DE GRÁFICOS ####
concat = alt.hconcat(
    ranged_dot_plot,
    line_plot,
    title=alt.Title("Brecha de género en la UE", fontSize=32, anchor='middle')
).resolve_scale(
    # Queremos que las escalas de colores y ejes sean independientes para mayor flexibilidad
    color='independent',
    y='independent'
).configure_axis(grid=False, domain=False)

display(concat)

alt.HConcatChart(...)

#### **Observaciones**

Se puede observar que si bien hay una brecha de género en la proporción de representantes en parlamentos de la Unión Europea, en la mayoría de los casos hay una tendencia a la disminución de la misma a medida que pasan los años.

### Violencia de género

A diferencia del análisis anterior, no encontramos un dataset que tenga en simultáneo data segmentada y que a la vez sea temporal. Por lo tanto armamos uno a partir de dos datasets que presentan información segmentada similar, pero para dos instancias temporales separadas: 2012 y 2021. Para más detalles respecto al procesamiento referirse a la sección correspondiente al principio de este reporte. Lo más relevante a comentar de esto es que los datasets tienen solamente dos segmentos etarios en común (18-29 años y total del rango etario), por lo que varias de las visualizaciones estarán limitadas por ello.

Para mostrar estos datos decidimos utilizar tres gráficos concatenados horizontalmente:
- *Normalized stacked bar chart*: relativamente simple pero efectivo en este caso que queremos transmitir proporciones binarias (Violencia sí/no). Además, porque nos permite resaltar el contraste entre los dos años analizados: 2012 y 2021, que muestran tendencias muy distintas (por las razones que fuera). En un simple bar chart esto último podría pasar desapercibido a menos que fijemos los valores de los ejes.
- *Radial chart*: escogimos este tipo de gráfico para representar las proporciones dado que efectivo para transmitir tamaños relativos. En este gráfico se pueden apreciar los segmentos etarios de cada subset (2012/2021).
- *Slope chart*: dado que contamos con solamente dos puntos temporales (2012 y 2021), el slope chart es un buen candidato para mostrar evolución temporal de la variable.

In [ ]:
# Seleccionamos los valores correspondientes a EU, países individuales de la EU
# Dado que hay dos series de datos para la EU: 28 países (hasta 2020) y 27 países (todo el rango), optamos por utilizar EU 27
source = df_violence[filter_eu(df_violence)]

# Valores que nos serań útiles
min_yr = source['Año'].min()
max_yr = source['Año'].max()

#### **Normalized Stacked-Bar Chart:** *Proporción de mujeres víctimas de violencia según país de la UE*

En este gráfico quisimos mostrar la proporción de mujeres víctimas de violencia. A modo de resaltar visualmente la diferencia entre el sí/no, optamos por un normalized stacked bar char en vez de un bar chart (filtrado por violencia sí).


Para facilitar la interacción del usuario con el gráfico agregamos las siguientes funcionalidades:
- Selección de años (radio button), que permite seleccionar dinámicamente el año de consulta, y a su vez la selección se refleja en el título del gráfico.
- Selección de rango etario (dropdown), que permite seleccionar dinámicamente el rango etario de consulta.
- Tooltips, que ayudan a la interpretación mostrando la información correspondiente a cada barra, como ser País, % violencia, gap, año.

Para facilitar la comparación entre países con valores similares, hemos ordenado los puntos de acuerdo a la proporción correspondiente al sí, en orden ascendiente.

En cuanto a los colores, decidimos dejar los valores default porque son relativamente neutrales y se distinguen muy bien uno del otro.



In [ ]:
### NORMALIZED STACKED-BAR CHART ###
# Definimos los selectores de nuestro gráfico

# Armamos un radio button que nos permita seleccionar dinámicamente el año
# The spacing will only show up in your IDE, not on this doc page
options = [min_yr, max_yr]
labels = [f"{option} " for option in options]

year_radio = alt.binding_radio(options=options, labels=labels, name='Seleccione el año: ')

# Objeto de selección de un punto en base al año
select_year = alt.selection_point(name='Year', fields=['Year'], bind=year_radio,
                                  value=max_yr)

# Objeto de selección de punto en base al país seleccionado en la leyenda
select_country = alt.selection_point(fields=['Entidad'], bind='scales')

# Selección de Rango Etario
ages_2021 = set(source[source['Año'] == 2021].Edad.unique())
ages_2012 = set(source[source['Año'] == 2012].Edad.unique())
age_range = list(set.intersection(ages_2021, ages_2012))

age_dropdown = alt.binding_select(options=age_range, name='Seleccione el rango de edades (años): ')
select_age = alt.selection_point(fields=['Edad'], bind=age_dropdown, value='Total', resolve='global')

# Título dinámico, dependiente del valor escogido de edad
title_bar = alt.Title(alt.expr(f'"Percepción violencia en mujeres, por país (Año: " + {select_year.name}.Year + ")"' ))

# Definimos el gráfico de barras normalizado
bar = alt.Chart(
    data=source,
    title=title_bar
    ).mark_bar().encode(
    x=alt.X('sum(Porcentaje)', title='Proporción de Mujeres (%)').stack("normalize"),
    y=alt.Y('Entidad:N', title='',
            # Ordenamos el eje Y de acuerdo al % de violencia
            sort=alt.EncodingSortField(field='PorcentajeSort', order='ascending')),
    color=alt.Color('Violencia:N', title='Violencia'),
    tooltip=['Entidad', 'Violencia', 'Porcentaje', 'Año']
).add_params(
    select_year,
    select_country,
    select_age
).transform_filter(
    select_year
).transform_filter(
    select_country
).transform_filter(
    select_age
)

bar

alt.Chart(...)

#### **Radial Chart:** *Proporción de mujeres víctimas de violencia según segmento etario*


En este gráfico quisimos mostrar la proporción de mujeres víctimas de violencia según el segmento etario. A modo de resaltar visualmente la diferencia entre proporciones, optamos por un radial chart en vez de un pie chart.

Para facilitar la interacción del usuario con el gráfico agregamos las siguientes funcionalidades:
- Selección de años (radio button), que permite seleccionar dinámicamente el año de consulta, y a su vez la selección se refleja en el título del gráfico.
- Selección de país (dropdown), que permite seleccionar dinámicamente el país de consulta, y a su vez la selección se refleja en el título del gráfico. Dado que para uno de los años no contamos con la data de la UE, queda excluida como opción.
- Tooltips, que ayudan a la interpretación mostrando la información correspondiente a cada barra, como ser País, % violencia, gap, año.

Los datos están ordenados por segmento etario. De esta forma es fácil comparar país/país y año/año.

En cuanto a los colores, decidimos dejar los valores default porque son relativamente neutrales y se distinguen muy bien uno del otro.



In [ ]:
### RADIAL CHART ###
# Filtrar datos cuando Violencia es 'Sí'
filtered_data = source[source['Violencia'] == 'Sí']

# Obtener los países que tienen datos para ambos años
countries_with_both_years = filtered_data.groupby('Entidad').filter(lambda x: set(x['Año']) == {2012, 2021})['Entidad'].unique()
# Ordenar los países alfabéticamente
sorted_countries = sorted(countries_with_both_years)

# Selección de País
country_dropdown = alt.binding_select(options=sorted_countries, name='Seleccione el país: ')
select_country = alt.selection_point(fields=['Entidad'], bind=country_dropdown, value=sorted_countries[0])

# Crear el gráfico base
base = alt.Chart(
    data=filtered_data,
    title=alt.Title(alt.expr(f'"Percepción violencia en mujeres, según segmento etario (" + {select_country.name}.Entidad + ", " + {select_year.name}.Year + ")"' ))
).encode(
    alt.Theta("Porcentaje:Q").stack(True),
    alt.Radius("Porcentaje:Q").scale(type="sqrt", zero=True, rangeMin=20),
    color="Edad:N"
).add_params(
    select_country,
    select_year
).transform_filter(
    select_country
).transform_filter(
    select_year
)

# Agregamos los siguientes parámetros para evitar que se rompa el radial plot al
# concatenarlo con otros gŕaficos
# Source: https://github.com/vega/altair/issues/3017
w = alt.param(name="width", value=300)
h = alt.param(name="height", value=300)

# Crear los arcos y etiquetas del gráfico
c1 = base.mark_arc(innerRadius=20, stroke="#fff")
c2 = base.mark_text(radiusOffset=10).encode(
    text="Porcentaje:Q"
).transform_calculate(label='datum.Porcentaje + " %"')

# Combinar los gráficos de arcos y etiquetas
radial_chart = c1 + c2

radial_chart

alt.LayerChart(...)

#### **Slope Chart:** *Evolución temporal proporción de mujeres víctimas de violencia en la UE*

En este gráfico quisimos mostrar la evolución temporal de la proporción de mujeres víctimas de violencia. Dado que sólo contamos con dos puntos de datos, optamos por un slope chart.

Para facilitar la interacción del usuario con el gráfico agregamos las siguientes funcionalidades:
- Selección de años (radio button), que permite seleccionar dinámicamente el año de consulta, y a su vez la selección se refleja en el título del gráfico.
- Selección de país (dropdown), que permite seleccionar dinámicamente el país de consulta, y a su vez la selección se refleja en el título del gráfico. Dado que para uno de los años no contamos con la data de la UE, queda excluida como opción.
- Tooltips, que ayudan a la interpretación mostrando la información correspondiente a cada barra, como ser País, % violencia, gap, año.

Los datos están ordenados por segmento etario. De esta forma es fácil comparar país/país y año/año.

En cuanto a los colores, decidimos dejar los valores default porque son relativamente neutrales y se distinguen muy bien uno del otro.


In [ ]:
#### SLOPE CHART ####
# Filtrar datos cuando Edad es '18-29' y 'Total', y solo cuando Violencia es 'Sí'
filtered_data = source[(source['Edad'].isin(age_range)) & (source['Violencia'] == 'Sí')]

# Título dinámico, dependiente del valor escogido de edad
title_slope = alt.TitleParams(
    alt.expr(f'"Evolución de violencia física y/o sexual contra las mujeres por país (Rango de edades: " + {select_age.name}.Edad + ")"' ),
)
# Asegurarse de que solo los países con datos para ambos años sean incluidos
countries_with_both_years = filtered_data.groupby('Entidad').filter(lambda x: x['Año'].nunique() == 2)

# Selector de país
select_country = alt.selection_multi(fields=['Entidad'], bind='legend')

# Armamos la base de nuestro chart, donde agregaremos nuestros selectores y filtros,
# los cuales aplicarán para el resto de las capas
chart = alt.Chart(
    data=countries_with_both_years
).add_selection(
    select_country
).transform_filter(
    select_country
).add_params(
    select_age
).transform_filter(
    select_age
)

# Capa de líneas
line_chart = chart.mark_line().encode(
    x=alt.X('Año:O', title='Año', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Porcentaje:Q', title='Proporción de Mujeres (%)'),
    color=alt.Color('Entidad:N', title='Seleccione país'),
    strokeOpacity=alt.condition(select_country, alt.value(0.8), alt.value(0.2)),
    strokeWidth=alt.condition(select_country, alt.value(2), alt.value(1)),
)

# Capa de círculos
point_chart = chart.mark_circle(size=120).encode(
    x='Año:O',
    y='Porcentaje:Q',
    color='Entidad:N',
    opacity=alt.condition(select_country, alt.value(0.8), alt.value(0.2)),
    tooltip=['Entidad', 'Porcentaje', 'Año']
)

# Combinar gráficos
slope_chart = alt.layer(line_chart, point_chart).properties(
    width=600,
    height=400,
    title=title_slope
)

slope_chart

/usr/local/lib/python3.10/dist-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/usr/local/lib/python3.10/dist-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)

#### Concatenación de gráficos

In [ ]:
#### CONCATENACIÓN DE GRÁFICOS ####
# Combinamos los gráficos
combined_chart = alt.hconcat(
    bar,
    radial_chart.add_params(w, h), # agregamos estos parámetros como workaround para que no se rompa el formato del radial chart
    slope_chart,
    title=alt.Title("Proporción de mujeres que han percibido violencia física y/o sexual en el último año", fontSize=32, anchor='middle')
).resolve_scale(
    color="independent",
    y='independent'
).configure_title(
    anchor='start'
)

combined_chart

alt.HConcatChart(...)

#### **Observaciones**

Si bien no hay suficientes datos temporales como para establecer una tendencia, se puede apreciar la notoria diferencia de respuestas clasificadas como violencia entre los años 2012 y 2021. Posibles causas que podrían explicar estas diferencias pueden ser mayor concienciación/apertura sobre el tema, cambios culturales, adoptación de nuevas legislaciones, etc.

De todas formas, se puede observar en ambos casos que la prevalencia de violencia se registra en todos los rangos etarios, con especial énfasis en los segmentos más jóvenes.

### Combinación con múltiples variables para comparar con la brecha de género (gender gap)

Se utilizan para este fin el dataset del Indice de desarrollo humano (HDI) y el GDP per capita.

Para esto usamos los datasets HDI y GDP per capita, y seleccionamos a partir del año 2003, ya que el dataset de parliament sólo contiene datos a partir de 2003.

Hacemos un simple **merge** entre estos 3 datasets con las variables que son de nuestro interés y eliminamos todo lo que sea innecesario.

In [ ]:
## Nos aseguramos que este dataset también tenga datos desde el mísmo periodo de tiempo.
df_parliament_2003_onwards = df_parliament[df_parliament['Año']>2002]

In [ ]:
### HACEMOS MERGE ENTRE LOS DATASETS GDP Y PARLIAMENT
df_gdp_n_parliament = pd.merge(df_gdp,df_parliament_2003_onwards, on=['Region','Año'], suffixes=("","_drop"))

# eliminamos columnas repetidas
columns_to_drop = [col for col in df_gdp_n_parliament if col.endswith("_drop")]
df_gdp_n_parliament.drop(columns=columns_to_drop, inplace = True)

# HACEMOS MERGE CON EL DATASET RESTANTE:
df_multiple_variables = pd.merge(df_gdp_n_parliament,df_hdi, on =['Region','Año'], suffixes=("","_drop"))

# eliminamos columnas repetidas
columns_to_drop = [col for col in df_multiple_variables if col.endswith("_drop")]
df_multiple_variables.drop(columns=columns_to_drop, inplace = True)


### Visualización:
Para comparar el Gap con variables adicionales, hacemos algo similar a lo que se hizo para las preguntas anteriores, creamos tres charts concatenados horizontalmente (Matriz de correlación y 2 diagramas de barras).

#### **Mapa de Calor:** *correlación entre variables HDI, GDP, y Gap*

Una matrix de correlación que varía la **intensidad de color**, entre más intenso (oscuro) más correlacionadas están las variables.

- Selección por dropdown: la vista inicial es sobre la Union Europea, pero permite seleccionar por país individualmente.  

- Tooltips: dinámico, que permite mostrar las variables en cuestión para la correlación.

El objetivo es responder:  

***Is there a correlation between gender gap and another variable(s) of the country such as
GDP , level of human development of their citizens, etc.?***

In [ ]:
# creamos función para determinar el coeficiente de correlación:

def coef_corr(data_frame, columns):
    corr = data_frame[columns].corr().reset_index().melt('index')
    corr.columns = ['Variable1', 'Variable2', 'Correlation']
    return corr

# definimos las columnas del dataset que vamos a comparar en la correlación
columns = ['GDP', 'Gap', 'HDI']

def coef_corr(data_frame, columns):
    corr = data_frame[columns].corr().reset_index().melt('index')
    corr.columns = ['Variable1', 'Variable2', 'Correlation']
    return corr

# hacemos un pre computo de la correlación por cada país.
corr_dfs = []
for country in df_multiple_variables['Region'].unique():
    filtered_df = df_multiple_variables[df_multiple_variables['Region'] == country]
    corr_df = coef_corr(filtered_df, columns)
    corr_df['Region'] = country
    corr_dfs.append(corr_df)

df_corr = pd.concat(corr_dfs)

# Selector de país
countries = sorted(df_corr['Region'].unique().tolist())
country_dropdown = alt.binding_select(options=countries + [None], labels=countries + ['European Union'], name='Seleccione país: ')
select_country = alt.selection_single(fields=['Region'], bind=country_dropdown)

# hacemos un mapa de calor para ver los niveles de correlación
heatmap = alt.Chart(df_corr).mark_rect().encode(
    x=alt.X('Variable1:O', title=None),
    y=alt.Y('Variable2:O', title=None),
    color=alt.Color('Correlation:Q'),
    tooltip=['Variable1', 'Variable2', 'Correlation']
).add_selection(
    select_country
).transform_filter(
    select_country
).properties(
    title='Correlation Matrix of GDP, Gap, and HDI',
    width=400,
    height=400
)

/usr/local/lib/python3.10/dist-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/usr/local/lib/python3.10/dist-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


#### **Diagramas de barras**: *con HDI y GDP por país*

Junto a esta matriz se encuentran dos diagramas de barras que muestran las variables **Indice de desarrollo humano (HDI)** y **Producto Interno Bruto (GDP)** per capita por cada país. Permitiendo facilmente al lector comprender el comportamiento de estas.  

En los dos casos, se decide trabajar con la totalidad de los datos (periodo completo) por lo cual hacemos una agregación de los datos por cada país, y **calculamos su media como valor total.**

##### **Bar Chart** con HDI por país

In [ ]:
#### Creación del Diagrama de barras de HDI por país ####

# Calculamos la media de HDI por cada país desde el año 2003
mean_hdi = df_multiple_variables.groupby('Region')['HDI'].mean().reset_index()

# ponemos la media de GDP en el eje horizontal y seleccionamos un color que vaya acorde con los otros diagramas de esta sección
ch1 = alt.Chart(mean_hdi).mark_bar(color='steelblue').encode(
    x=alt.X('HDI:Q', title='Mean HDI'),
    y=alt.Y('Region:N', title='Countries')
)

# añadimos texto a las variables y usasmos el dx en 3 para que queden por fuera de la barra y faciliten lectura.
ch2 = alt.Chart(mean_hdi).mark_text(
    align='left',
    baseline='middle',
    dx=3,
    color='black'
).encode(
    x=alt.X('HDI:Q'),
    y=alt.Y('Region:N'),
    text=alt.Text('HDI:Q', format='.2f')
)

# Combinamos las barras con el texto en un solo diagrama.
hdi_media = alt.layer(ch1, ch2).properties(
    title=alt.TitleParams('Mean HDI per Country', subtitle='Mean over the period between 2003-2022'),
    width=400,
    height=400
)

#### Creación del Diagrama de barras de GDP por país ####

# Calculamos la media de GDP por cada país desde el año 2003
mean_gdp = df_multiple_variables.groupby('Region')['GDP'].mean().reset_index()

# ponemos la media de GDP en el eje horizontal y seleccionamos un color que vaya acorde con los otros diagramas de esta sección
gdp_media = alt.Chart(mean_gdp).mark_bar(color='steelblue').encode(
    x=alt.X('GDP:Q', title='Mean GDP (€)'),
    y=alt.Y('Region:N', title='Countries')
).properties(
    title=alt.TitleParams('Mean GDP per Country', subtitle='Mean over the period between 2003-2022'),
    width=400,
    height=400
)

##### **Bar Chart** con GDP por país



In [ ]:
# Bar chart
ch1_GDP = alt.Chart(df_multiple_variables).mark_bar(color='steelblue').encode(
    alt.X('GDP:Q', axis=None),
    y=alt.Y('Region:N', title='Countries')
).properties(
    title='GDP per country')


#### **Cuadro combinado**: *Concatenación de los 3 cuadros*

In [ ]:
### COMBINAMOS LOS CHARTS

combined_chart = alt.hconcat(heatmap, hdi_media, gdp_media,
                             title=alt.Title("Comparación de relación entre Gap con HDI y GDP per capita", fontSize=32, anchor='middle')
).resolve_scale(
    color='independent',
    y='independent'
    ).configure_view(
    strokeWidth=0, strokeOpacity=0,

)

# mostramos el chart combinado
combined_chart

alt.HConcatChart(...)

#### **Visualizaciones múltiples pequeñas con filtro en país**

***Show the comparison between gender gap and the variables analyzed in the previous point.***  

Decidimos hacer dos charts que miden cada una de las variables (el GDP y el HDI) y **muestran la magnitud del gap de acuerdo a la intensidad del color** (rojo, gap más alto, verde, gap más bajo).

- Selección de país (dropdown): permite seleccionar el país de interés, y la Union Europea en su totalidad.

Nos pareció visualmente más explicativo hacer esto por año para que el efecto de las variables adicionales sea más visible, y tratar de ver si hay una tendencia a la mejoría o un deterioro en el mencionado Gap.

In [ ]:
# hacemos un dropdown para seleccionar el país
countries = sorted(df_multiple_variables['Region'].unique().tolist())
dropdown_country = alt.binding_select(options=countries + [None],
                                      labels=countries + ['European Union'], name='Seleccione pais: ')
select_country = alt.selection_single(fields=['Region'], bind=dropdown_country, name='Seleccione pais: ')

# creamos el chart de base
base = alt.Chart(df_multiple_variables).add_selection(
    select_country
).transform_filter(
    select_country
)

# GDP vs Año, con el color representando el gap
gdp_vs_year = base.mark_circle(size=100).encode(
    x=alt.X('Año:O', title='Year'),
    y=alt.Y('GDP:Q', title='GDP'),
    color=alt.Color('Gap:Q', title='Gap', scale=alt.Scale(range=['green', 'red'], domain=[0,100])),
    tooltip=['Region', 'Año', 'GDP', 'Gap', 'HDI']
).properties(
    title=alt.TitleParams('GDP vs Year', subtitle='Color intensity shows Gap evolution'),
    width=600,
    height=400
)

# GDP vs Año con el color representando el gap
hdi_vs_year = base.mark_circle(size=60).encode(
    x=alt.X('Año:O', title='Year'),
    y=alt.Y('HDI:Q', title='HDI', scale=alt.Scale(domain=[0.7, 1] )),
    color=alt.Color('Gap:Q', title='Gap', scale=alt.Scale(range=['green', 'red'], domain=[0,100] )),
    tooltip=['Region', 'Año', 'GDP', 'Gap', 'HDI']
).properties(
    title=alt.TitleParams('HDI vs Year', subtitle='Color intensity shows Gap evolution'),
    width=600,
    height=400
)

# combinamos los plots en pequeños multiples:
combinado_multiple = alt.hconcat(
    gdp_vs_year,
    hdi_vs_year,
    title=alt.Title("Comparación Gap vs HDI y GDP per Capita por país a través del tiempo", fontSize=32, anchor='middle')
).resolve_scale(
    color='independent'
)

# mostramos los plots concatenados horizontalmente
combinado_multiple

/usr/local/lib/python3.10/dist-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/usr/local/lib/python3.10/dist-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.HConcatChart(...)

#### **Observaciones**
A pesar de que no aplica para todos los países, en general, para la Unión Europea hay una tendencia de incremento de tanto el HDI como GDP, que se podría traducir en una disminución del gap (brecha de género). Esta correlación es más evidente en el caso del HDI.

## Guardado de datasets como csv:

In [ ]:
datasets = {
    'GDP': df_gdp,
    'HDI': df_hdi,
    'Parliament': df_parliament,
    'Violence': df_violence
}

for name, dataset in datasets.items():
  dataset.to_csv(f"{name}_processed.csv", index = False)